In [ ]:

import pandas as pd
import langid
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import en_core_web_sm
import spacy_fastlang
import regex

nlp = en_core_web_sm.load()
nlp.add_pipe("language_detector")



In [ ]:
#setting up necessary proccessing things

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


df = pd.read_csv('data/goodreads_fiction.csv')
df['Review'] = df['Review'].str.replace("121", " ")
df['Review'] = df['Review'].str.replace("\n", " ")

df


In [ ]:
def remove_non_latin_letters(text): #removes anything other than non latin characters
    cleaned_text = ''.join(c if (regex.match(r'\p{IsLatin}', c) or c.isdigit() or c.isspace() or (c == "'" and i < len(text) - 1 and regex.match(r'\p{IsLatin}', text[i+1]))) else ' ' for i, c in enumerate(str(text)))
    return ' '.join(cleaned_text.split())



def chunk_text (text, chunk_size=75):
    words = text.split()
    for i in range (0, len(words), chunk_size):
        yield " ".join(words[i:i+chunk_size])

def check_en (text):
    doc = nlp(text)
    return doc._.language == 'en'

def process_item(text):
    processed_text = []
    for chunk in chunk_text(text):
        if check_en(chunk):
            processed_text.append(chunk)
    return ' '.join(processed_text)

def remove_stopwords_and_lemmatize(text):
    tokens = word_tokenize(text)
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.lower() not in stop_words and word.isalpha()]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


In [ ]:
#remove all none letters from reviews synopsis and genres
columns_to_clean = ['Synopsis','Review','Genres']
df["Raw Synopsis"] = df["Synopsis"]
for column in columns_to_clean:
    df[column] = df[column].apply(remove_non_latin_letters)
    
for column in columns_to_clean:
    try:
        df[column] = df[column].str.lower()
    except:
        print ("error")


#remove all the failed data from webscraping
df_filtered = df[~df.eq("errorlololol").any(axis=1)]
df = df_filtered

df
columns_to_check = ['Review', 'Genres', 'Synopsis']

for col in columns_to_check:
    if df[col].isna().any():
        print(f"'{col}' column contains NoneType or NaN values.")
    else:
        print(f"'{col}' column does not contain NoneType or NaN values.")




df

In [ ]:
columns_to_proccess = ['Synopsis', 'Review']


    
df['Proccessed Synopsis'] = df['Synopsis'].apply(process_item)
df['Proccessed Review'] = df['Review'].apply(process_item)

df['Proccessed Synopsis'] = df['Proccessed Synopsis'].apply(remove_stopwords_and_lemmatize)
df['Proccessed Review'] = df['Proccessed Review'].apply(remove_stopwords_and_lemmatize)
    
df
    

In [ ]:

print(df["Proccessed Synopsis"])

for hello in df['Proccessed Synopsis']:
    if hello == None:
        print("there is NaN type in proccessed synopsis")
    else:
        print("false")
#1241 should be the right answer

In [ ]:
df.to_csv('data\cleaned_fic_file', index=False)